In [74]:
import pandas as pd
import numpy as np
import json
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import rdChemReactions
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.Draw import IPythonConsole
from IPython.display import SVG, display
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
from rdkit.Chem import MACCSkeys
import pandas as pd
import rdchiral
import itertools
import pickle
import sys
from tqdm import tqdm 
from rdchiral.main import rdchiralRun, rdchiralRunText, rdchiralReaction, rdchiralReactants
import cobra
from itertools import chain, combinations
import sys
import multiprocessing
from functools import partial
import matplotlib.pyplot as plt


sys.path.append('../../retrosynthesis/')
from common import *

### input and output

In [75]:
#input
yeast870_path = '../../../Data/model/yeast-GEM.yml'
yeast8U_path = '../../../Data/model/Yeast-MetaTwin.yml'
metabolites_info_to_GEM_path = '../../../Results/not_lipid/top50_0.3_re/metabolites_info_to_GEM_del_top50_0.3.csv'
mnxreac_smile_atom_mapping_rules_file_path = '../../../Results/rules/MNXreaction_smiles_atommap_rules_filter.csv'

In [76]:
yeast8 = cobra.io.load_yaml_model(yeast870_path)
yeast8U = cobra.io.load_yaml_model(yeast8U_path)

In [77]:
metabolites_info_to_GEM = pd.read_csv(metabolites_info_to_GEM_path)
metabolites_info_to_GEM

,new_met_smiles,sim_smile,ID,compartment
0,CCCCCCCCCCCCCC(=O)[O-],CCCCCCCCCCCCCC(=O)O,s_1161,c
1,NC(Cc1cnc[nH]1)C(=O)O,NC(Cc1cnc[nH]1)C(=O)O,s_1006,c
2,CSCCC(=O)C(=O)COP(=O)(O)O,CSCCC(=O)C(=O)COP(=O)(O)O,s_0311,c
3,CC(N)C(=O)NCC(=O)[O-],CC(N)C(=O)NCC(=O)O,s_4171,c
4,O=C(O)Cc1ccccc1,O=C(O)Cc1ccccc1,s_1321,c
...,...,...,...,...
3466,Nc1nc(=O)c2c([nH]1)NCC(CNc1ccc(C(=O)NC(CCC(=O)...,Nc1nc2c(c(=O)[nH]1)N=C(CNc1ccc(C(=O)NC(CCC(=O)...,s_0625,c
3467,CCCCCCCCCCCCCCCCCC(=O)OCC(O)CO,,sn_323,c
3468,O=C([O-])c1cc(=O)[nH]c(=O)n1C1OC(COP(=O)([O-])...,O=C(O)c1cc(=O)[nH]c(=O)n1C1OC(COP(=O)(O)O)C(O)C1O,s_1270,c
3469,O=C([O-])C=C(CCC(=O)[O-])C(=O)[O-],O=C(O)C=C(CCC(=O)O)C(=O)O,s_0454,m


# native

In [78]:
# L-glutamate --> L-glutamic 5-semialdehyde
# L-glutamate:s_0991
# L-glutamic 5-semialdehyde:s_0997
# P07275  YHR037W
# H2O + L-glutamate 5-semialdehyde + NAD+ = 2 H+ + L-glutamate + NADH
# EC:1.2.1.88 (UniProtKB | ENZYME| Rhea)

for i in yeast8.reactions:
    if 's_0991' in i.reaction and 's_0997' in i.reaction:
        print(i.id,i.reaction)
        print({k.id+' ['+ k.name+']':v for k,v in i.metabolites.items()})
print('===========')
for i in yeast8U.reactions:
    if 's_0991' in i.reaction and 's_0997' in i.reaction:
        print(i.id,i.reaction)
        print({k.id+' ['+ k.name+']':v for k,v in i.metabolites.items()})
# s_0794 + s_0991 + s_1212 --> s_0803 + s_0997 + s_1207

r_0819 s_0180 + s_1266 --> s_0991 + s_0997
{'s_0180 [2-oxoglutarate]': -1, 's_0991 [L-glutamate]': 1, 's_0997 [L-glutamic 5-semialdehyde]': 1, 's_1266 [ornithine]': -1}
r_0819 s_0180 + s_1266 --> s_0991 + s_0997
{'s_0180 [2-oxoglutarate]': -1, 's_0991 [L-glutamate]': 1, 's_0997 [L-glutamic 5-semialdehyde]': 1, 's_1266 [ornithine]': -1}
rxn651 s_0794 + s_0991 + s_1212 --> s_0803 + s_0997 + s_1207
{'s_0794 [H+]': -1, 's_0803 [H2O]': 1, 's_0991 [L-glutamate]': -1, 's_0997 [L-glutamic 5-semialdehyde]': 1, 's_1207 [NADP(+)]': 1, 's_1212 [NADPH]': -1}
rxn1649 s_0794 + s_0991 + s_1203 --> s_0803 + s_0997 + s_1198
{'s_0794 [H+]': -1, 's_0803 [H2O]': 1, 's_0991 [L-glutamate]': -1, 's_0997 [L-glutamic 5-semialdehyde]': 1, 's_1198 [NAD]': 1, 's_1203 [NADH]': -1}
rxn2032 s_0145 + s_0991 --> s_0997 + s_4173
{'s_0145 [2-acetamido-5-oxopentanoate]': -1, 's_0991 [L-glutamate]': -1, 's_0997 [L-glutamic 5-semialdehyde]': 1, 's_4173 [N-acetyl-L-glutamate]': 1}


In [79]:
[x.id for x in yeast8U.reactions.get_by_id('rxn651').genes]
# YHR037W

['YBR006W',
 'YPL098C',
 'YER073W',
 'YGR192C',
 'YOR323C',
 'YMR110C',
 'YDR158W',
 'YJR009C',
 'YJL052W',
 'YHR037W']

In [80]:
# kynurenate + L-glutamic --> L-kynurenine + 2-oxoglutarate
# s_2763(kynurenic acid/kynurenate ) +  s_0803 +  s_0991(L-glutamate) => s_1020(L-kynurenine) +  s_0180(2-oxoglutarate) 
# kynurenate:s_2763
# L-glutamic:s_0991
# s_0803
# L-kynurenine:s_1020
# 2-oxoglutarate:s_0180
# P38840_1  s_2763 +  s_0803 +  s_0991 => s_1020 +  s_0180
# 2-oxoglutarate + L-kynurenine = H2O + kynurenate + L-glutamate
# EC:2.6.1.7 (UniProtKB | ENZYME
# | Rhea)
# YHR137W

for i in yeast8.reactions:
    if 's_2763' in i.reaction and 's_1020' in i.reaction and 's_0180' in i.reaction:
        print(i.id,i.reaction)
        print({k.id+' ['+ k.name+']':v for k,v in i.metabolites.items()})
print('===========')
for i in yeast8U.reactions:
    if 's_2763' in i.reaction and 's_1020' in i.reaction and 's_0180' in i.reaction:
        print(i.id,i.reaction)
        print({k.id+' ['+ k.name+']':v for k,v in i.metabolites.items()})
#s_0803 + s_0991 + s_2763 --> s_0180 + s_1020

rxn557 s_0803 + s_0991 + s_2763 --> s_0180 + s_1020
{'s_0180 [2-oxoglutarate]': 1, 's_0803 [H2O]': -1, 's_0991 [L-glutamate]': -1, 's_1020 [L-kynurenine]': 1, 's_2763 [kynurenic acid]': -1}


In [81]:
[x.id for x in yeast8U.reactions.get_by_id('rxn557').genes]
# YHR137W

['YHR137W',
 'YKL106W',
 'YOR184W',
 'YMR084W',
 'YJR148W',
 'YGL202W',
 'YDR111C',
 'YLR201C',
 'YHR208W',
 'YMR085W',
 'YER137C',
 'YNR033W',
 'YGR243W',
 'YER152C',
 'YJL060W',
 'YLR089C']

In [82]:
# L-lysine --> Cadaverine  ### YKL184W P08432 
# L-lysine:s_1025
# Cadaverine:s_4299

for i in yeast8.reactions:
    if 's_1025' in i.reaction and 's_4299' in i.reaction:
        print(i.id,i.reaction)
        print({k.id+' ['+ k.name+']':v for k,v in i.metabolites.items()})
print('===========')
for i in yeast8U.reactions:
    if 's_1025' in i.reaction and 's_4299' in i.reaction:
        print(i.id,i.reaction)
        print({k.id+' ['+ k.name+']':v for k,v in i.metabolites.items()})      
#s_0794 + s_1025 --> s_0456 + s_4299

rxn2384 s_0794 + s_1025 --> s_0456 + s_4299
{'s_0456 [carbon dioxide]': 1, 's_0794 [H+]': -1, 's_1025 [L-lysine]': -1, 's_4299 [cadaverine]': 1}


In [83]:
[x.id for x in yeast8U.reactions.get_by_id('rxn2384').genes] 
#YKL184W

['YDR380W',
 'YLR134W',
 'YDL080C',
 'YLR044C',
 'YLL028W',
 'YER010C',
 'YGR087C',
 'YDR539W',
 'YOR128C',
 'YOL052C']

In [84]:
# L-homocysteine + S-methyl-L-methionine = H+ + 2 L-methionine  #Q08985 YPL273W
# L-methionine:s_1029
# S-methyl-L-methionine:s_1424
# L-homocysteine:s_1012

for i in yeast8.reactions:
    if 's_1029' in i.reaction and 's_1424' in i.reaction and 's_1012' in i.reaction:
        print(i.id,i.reaction)
        print({k.id+' ['+ k.name+']':v for k,v in i.metabolites.items()}) 
print('===========')
for i in yeast8U.reactions:
    if 's_1029' in i.reaction and 's_1424' in i.reaction and 's_1012' in i.reaction:
        print(i.id,i.reaction)
        print({k.id+' ['+ k.name+']':v for k,v in i.metabolites.items()})   

#s_1012 + s_1424 --> s_0794 + 2 s_1029   

rxn1976 s_1012 + s_1424 --> s_0794 + 2 s_1029
{'s_0794 [H+]': 1, 's_1012 [L-homocysteine]': -1, 's_1029 [L-methionine]': 2, 's_1424 [S-methyl-L-methionine]': -1}


In [85]:
[x.id for x in yeast8U.reactions.get_by_id('rxn1976').genes] 
#YPL273W

['YER175C',
 'YKR069W',
 'YOL125W',
 'YBR061C',
 'YML110C',
 'YLR137W',
 'YNL092W',
 'YDR435C',
 'YGL136C',
 'YLR285W',
 'YIL064W',
 'YOL141W',
 'YLR172C']

# ESP

In [86]:
rxndb_total_info_to_model_del2_reaction_path = '../../../Results/not_lipid/top50_0.3_re/rxndb_total_info_to_model_del2_reaction_top50_0.3.csv'
metabolites_info_to_GEM_del_path = '../../Results/not_lipid/top50_0.3_re/metabolites_info_to_GEM_del_top50_0.3.csv'

In [87]:
rxndb_total_info_to_model_del2_reaction = pd.read_csv(rxndb_total_info_to_model_del2_reaction_path)
print(rxndb_total_info_to_model_del2_reaction.shape)
rxndb_total_info_to_model_del2_reaction.head(3)

(3281, 8)


,NO,EC_number,templateID,rxn_smiles_basic,rxn_smiles_final,GPR,equation_dict,equation
0,rxn1,1.14.13,MNXR149799,CCCCCCCCCCCC(=O)[O-].O=O>>CCCCCCCCCC(O)CC(=O)[...,CCCCCCCCCCCC(=O)[O-].O=O.[H+].NC(=O)C1=CN([C@@...,(YDR402C) or (YDR534C) or (YAL064W) or (YJL151...,"{'s_1065': -1, 's_1275': -1, 's_0794': -1, 's_...",s_1065 + s_1275 + s_0794 + s_1212 => sn_2...
1,rxn2,1.14.13,MNXR149799,CCCCCCCCCC(=O)[O-].O=O>>CCCCCCCC(O)CC(=O)[O-].O,CCCCCCCCCC(=O)[O-].O=O.[H+].NC(=O)C1=CN([C@@H]...,(YDR402C) or (YDR534C) or (YAL064W) or (YJL151...,"{'s_0595': -1, 's_1275': -1, 's_0794': -1, 's_...",s_0595 + s_1275 + s_0794 + s_1212 => sn_1...
2,rxn3,1.14.13,MNXR149799,CCCCCCCC(=O)[O-].O=O>>CCCCCC(O)CC(=O)[O-].O,CCCCCCCC(=O)[O-].O=O.[H+].NC(=O)C1=CN([C@@H]2O...,(YDR402C) or (YDR534C) or (YAL064W) or (YJL151...,"{'s_1248': -1, 's_1275': -1, 's_0794': -1, 's_...",s_1248 + s_1275 + s_0794 + s_1212 => sn_2...


In [88]:
rxndb_total_info_to_model_del2_reaction[rxndb_total_info_to_model_del2_reaction['NO']=='rxn2384']['GPR'].to_list()[0]

'(YLL028W) or (YOL052C) or (YLR134W) or (YGR087C) or (YDL080C) or (YLR044C) or (YDR380W) or (YOR128C) or (YDR539W) or (YER010C)'

In [89]:
rxndb_total_info_to_model_del2_reaction[rxndb_total_info_to_model_del2_reaction['NO']=='rxn1976']['rxn_smiles_final'].to_list()[0]

'C[S+](C)CCC([NH3+])C(=O)[O-].[NH3+]C(CCS)C(=O)[O-]>>CSCCC([NH3+])C(=O)[O-].CSCCC([NH3+])C(=O)[O-].[H+]'

In [90]:
cut_off = 0.3
num = 50
num_process = 50
cut_off_path = f'../../../Results/not_lipid/top{num}_{cut_off}_re/'
yeast870_path =  '../../../Data/model/yeast-GEM902.yml'
miss_met_id_file = '../../../Data/model/miss_met_id.json'
metabolites_info_to_GEM_path = cut_off_path + f'metabolites_info_to_GEM_top{num}_{cut_off}.csv'
rxndb_total_info_to_model_path = cut_off_path + f'rxndb_total_info_to_model_top{num}_{cut_off}.csv'
yeast_genome_file = '../../../Data/Saccharomyces_cerevisiae.fasta'
rxndb2model_compouns_protein_pair_df_file = '../../../Results/compound_protein/rxndb2model_compouns_protein_pair_df.csv'
rxndb2model_compouns_protein_pair_df_drop_esp_result_file = '../../../Results/compound_protein/rxndb2model_compouns_protein_pair_df_drop_esp.csv'

In [91]:
rxndb_total_info_to_model = pd.read_csv(rxndb_total_info_to_model_path)
print(rxndb_total_info_to_model.shape)
rxndb_total_info_to_model.head(3)

(7889, 8)


,NO,EC_number,templateID,rxn_smiles_basic,rxn_smiles_final,GPR,equation_dict,equation
0,rxn19747,1.14.13,MNXR180996_reverse,COc1cc(CC(=O)[O-])ccc1O.O>>COc1cc(C(C)=O)ccc1O...,COc1cc(CC(=O)[O-])ccc1O.O.NC(=O)c1ccc[n+]([C@@...,(YBL098W) or (YNR053C) or (YCL059C) or (YHR176...,"{'sn_256': -1, 's_0803': -1, 's_1198': -1, 'sn...",sn_256 + s_0803 + s_1198 => sn_226 + s_12...
1,rxn19748,1.14.13,MNXR149799,CCCCCCCCCCCC(=O)[O-].O=O>>CCCCCCCCCC(O)CC(=O)[...,CCCCCCCCCCCC(=O)[O-].O=O.[H+].NC(=O)C1=CN([C@@...,(YBL098W) or (YNR053C) or (YCL059C) or (YHR176...,"{'s_1065': -1, 's_1275': -1, 's_0794': -1, 's_...",s_1065 + s_1275 + s_0794 + s_1212 => sn_2...
2,rxn19749,1.14.13,MNXR149799,CCCCCCCCCC(=O)[O-].O=O>>CCCCCCCC(O)CC(=O)[O-].O,CCCCCCCCCC(=O)[O-].O=O.[H+].NC(=O)C1=CN([C@@H]...,(YBL098W) or (YNR053C) or (YCL059C) or (YHR176...,"{'s_0595': -1, 's_1275': -1, 's_0794': -1, 's_...",s_0595 + s_1275 + s_0794 + s_1212 => sn_1...


In [92]:
rxndb_total_info_to_model[rxndb_total_info_to_model['rxn_smiles_final']=='C[S+](C)CCC([NH3+])C(=O)[O-].[NH3+]C(CCS)C(=O)[O-]>>CSCCC([NH3+])C(=O)[O-].CSCCC([NH3+])C(=O)[O-].[H+]']['NO'].to_list()

['rxn9778']

In [93]:
rxndb2model_compouns_protein_pair_df_drop_esp_result = pd.read_csv(rxndb2model_compouns_protein_pair_df_drop_esp_result_file)
rxndb2model_compouns_protein_pair_df_drop_esp_result.head(3)

,NO,Smiles,gene,Sequence,ESP_Prediction,#metabolite in training set
0,rxn19747,COc1cc(CC(=O)[O-])ccc1O,YLL057C,MSPAAAQTAIPLPSTDLPVKIITNGLKNLNYTSKQGYGNFDTHFYD...,0.147008,0
1,rxn19747,COc1cc(CC(=O)[O-])ccc1O,YDR402C,MELLKLLCLILFLTLSYVAFAIIVPPLNFPKNIPTIPFYVVFLPVI...,0.122151,0
2,rxn19747,COc1cc(CC(=O)[O-])ccc1O,YOL114C,MTTLMGKFKLTGRSPLFVLQPMLHCKKQQFVEEAVRLISNKKIGKK...,0.005617,0


In [94]:
rxndb2model_compouns_protein_pair_df_drop_esp_result[
    rxndb2model_compouns_protein_pair_df_drop_esp_result['NO'] == 'rxn9778'
].sort_values(by='ESP_Prediction', ascending=False)

,NO,Smiles,gene,Sequence,ESP_Prediction,#metabolite in training set
105988,rxn9778,[NH3+]C(CCS)C(=O)[O-],YOL141W,MKNLTTIKQTNKNVKQERRKKYADLAIQGTNNSSIASKRSVELLYL...,0.600925,33
105956,rxn9778,[NH3+]C(CCS)C(=O)[O-],YOL125W,MLQDNNGPAVKRAKPSERLQCEYFMEKKKRRCGMTRSSQNLYCSEH...,0.556021,33
105969,rxn9778,[NH3+]C(CCS)C(=O)[O-],YLR285W,MSDIESLGEAAGLFEEPEDFLPPPPKPHFAEYQRSHITKESKSDVK...,0.548795,33
105933,rxn9778,[NH3+]C(CCS)C(=O)[O-],YML110C,MLISSRIVRSSLVNVPLRLSRCFTQAHRACKEEEVNSPLSSAAEQP...,0.514549,33
105965,rxn9778,[NH3+]C(CCS)C(=O)[O-],YLR137W,MAFKLWLLDEETIYEHVFERYTQLEGQSGKLAQDLGIQDRRGGVLE...,0.512751,33
...,...,...,...,...,...,...
105953,rxn9778,[NH3+]C(CCS)C(=O)[O-],YGR006W,MDLDLASILKGEISKKKKELANSKGVQPPCTEKFQPHESANIDETP...,0.013898,33
105876,rxn9778,C[S+](C)CCC([NH3+])C(=O)[O-],YHL036W,MEPLLFNSGKANPSQDVFIDVEVGDITTKYGSTNTGSFSSMDTVEA...,0.012303,2
105881,rxn9778,C[S+](C)CCC([NH3+])C(=O)[O-],YGR006W,MDLDLASILKGEISKKKKELANSKGVQPPCTEKFQPHESANIDETP...,0.012078,2
105869,rxn9778,C[S+](C)CCC([NH3+])C(=O)[O-],YDR105C,MGAVISLPVSMAGSFVASCFGGCCSNLVTKTASSLGSSSLGTRLLY...,0.008996,2
